# Deep Learning with PyTorch


Author: [Anand Saha](http://teleported.in/)

### 5. Autoencoder: denoising images

In [ ]:
import torch
import torch.cuda as cuda
import torch.nn as nn
import matplotlib.pyplot as plt
import numpy as np

from torch.autograd import Variable

# Torchvision module contains various utilities, classes, models and datasets 
# used towards computer vision usecases
from torchvision import datasets
from torchvision import transforms

# Functional module contains helper functions
import torch.nn.functional as F

In [ ]:
# Mean and standard deviation of all the pixels in the FMNIST dataset
# It is 0.5 since these are PIL images
mean_gray = 0.5
stddev_gray = 0.5

#transform=transforms.Compose([transforms.ToTensor(), transforms.Normalize((mean_gray,), (stddev_gray,))])
transform=transforms.Compose([transforms.ToTensor()])

cifar10_train = datasets.CIFAR10('./data.cifar10', train=True, download=True, transform=transform)
cifar10_valid = datasets.CIFAR10('./data.cifar10', train=False, download=True, transform=transform)

In [ ]:
def show_img(img, title):
    img = img.swapaxes(0, 1)
    img = img.swapaxes(1, 2)
    print(img.shape)
    fig = plt.figure(figsize=(2, 2))
    plt.imshow(img)
    plt.title(title)
    plt.show()

In [ ]:
show_img(cifar10_train[4][0].numpy(), 'Orig')

In [ ]:
batch_size = 500 # Reduce this if you get out-of-memory error

In [ ]:
cifar10_train_loader = torch.utils.data.DataLoader(cifar10_train, batch_size=batch_size, shuffle=True, num_workers=1)
cifar10_valid_loader = torch.utils.data.DataLoader(cifar10_valid, batch_size=batch_size, shuffle=True, num_workers=1)

In [ ]:
def weights_init(m):
    classname = m.__class__.__name__
    if classname.find('Conv') != -1:
        print('Conv Weight Init ..')
        m.weight.data.normal_(0.0, 0.02)
        m.bias.data.fill_(0.0001)
    elif classname.find('BatchNorm') != -1:
        m.weight.data.normal_(1.0, 0.02)
        m.bias.data.fill_(0)

In [ ]:
class DenoisingAutoencoder(nn.Module):
    
    def __init__(self):
    
        super(DenoisingAutoencoder, self).__init__()
                                         # 32 x 32 x 3 (input)

        self.conv1e = nn.Conv2d(3, 24, 3, padding=2)                   # 30 x 30 x 24
        self.conv2e = nn.Conv2d(24, 48, 3, padding=2)                  # 28 x 28 x 48
        self.conv3e = nn.Conv2d(48, 96, 3, padding=2)                  # 26 x 26 x 96
        self.conv4e = nn.Conv2d(96, 128, 3, padding=2)                 # 24 x 24 x 128
        self.conv5e = nn.Conv2d(128, 256, 3, padding=2)                # 22 x 22 x 256
        self.mp1e   = nn.MaxPool2d(2, return_indices=True)  # 11 x 11 x 256

        self.mp1d = nn.MaxUnpool2d(2)
        self.conv5d = nn.ConvTranspose2d(256, 128, 3, padding=2)
        self.conv4d = nn.ConvTranspose2d(128, 96, 3, padding=2)
        self.conv3d = nn.ConvTranspose2d(96, 48, 3, padding=2)
        self.conv2d = nn.ConvTranspose2d(48, 24, 3, padding=2)
        self.conv1d = nn.ConvTranspose2d(24, 3, 3, padding=2)
    
    def forward(self, x):
        
        x = self.conv1e(x)
        x = self.conv2e(x)
        x = self.conv3e(x)
        x = self.conv4e(x)
        x = self.conv5e(x)
        x, i = self.mp1e(x)
        x = self.mp1d(x, i)
        x = self.conv5d(x)
        x = self.conv4d(x)
        x = self.conv3d(x)
        x = self.conv2d(x)
        x = self.conv1d(x)
        
        return x


In [ ]:
"""
autoencoder = DenoisingAutoencoder().cuda()

for image,label in cifar10_train_loader:
    image = Variable(image).cuda()
    output = autoencoder(image)
    print(output.size())
    break
    
show_img(image.data.cpu().numpy()[1], 'sadf')
show_img(output.data.cpu().numpy()[1], 'sadf')
"""

In [ ]:
learning_rate = 0.001
noise_level = 0.1

In [ ]:
autoencoder = DenoisingAutoencoder().cuda()

In [ ]:
parameters = list(autoencoder.parameters())
loss_func = nn.MSELoss()
optimizer = torch.optim.Adam(parameters, lr=learning_rate)
#optimizer = torch.optim.SGD(parameters, lr=learning_rate, momentum=0.99, nesterov=True, dampening=0.0)

In [ ]:
import time

In [ ]:
for i in range(50):
    
    """
    if i % 20 == 0:
        print("Setting learning rate to ", decay(i+1))
        for param_group in optimizer.param_groups:
            param_group['lr'] = decay(i+1)
    """
            
    for image, label in cifar10_train_loader:
        
        noise = torch.randn(batch_size, 3, 32, 32) * noise_level
        image_n = torch.add(image, noise)
        
        image = Variable(image).cuda()
        image_n = Variable(image_n).cuda()

        optimizer.zero_grad()
        output = autoencoder(image_n)
        
        loss = loss_func(output, image)
        loss.backward()
        optimizer.step()
        
    
    # check image with noise and denoised image\n# Better image if you train more or upgrade the model\n
    img = image[0].cpu()
    input_img = image_n[0].cpu()
    output_img = output[0].cpu()

    origin = img.data.numpy()
    inp = input_img.data.numpy()
    out = output_img.data.numpy()

    
    show_img(origin, 'Orig')
    show_img(inp, 'With noise')
    show_img(out, 'Denoised')
    
    print(loss, i)
    print("Sleeping ...")
    time.sleep(10)
    print("Waking up ...")
    if False and i > 0 and i % 10 == 0:
        print("Sleeping ...")
        time.sleep(20)
        
    
